In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

### RFM model metrics:

**Recency** = days since last customer transaction <br>
**Frequency** = number of transactions in the last 12 month or some other period <br>
**Monetary Value** = total spend in the last 12 month or some other period <br>

Even though 12 month is the standard value, we can choose other period based on the business model. Here we will analyze data for just one year - 2011.

## STEP 1: Load and Explore Data

In [2]:
retail = pd.read_csv("../cohort_analysis/online_retail_II.csv")

In [3]:
retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [4]:
retail.shape

(1067371, 8)

In [5]:
print("number of unique customers:", retail['Customer ID'].nunique())

number of unique customers: 5942


In [6]:
#checking duplicates
print(f"there are {retail.duplicated().sum()} duplicated rows")

there are 34335 duplicated rows


In [7]:
#viewing duplicated rows
retail[retail.duplicated()].head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
371,489517,21912,VINTAGE SNAKES & LADDERS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
383,489517,22130,PARTY CONE CHRISTMAS DECORATION,6,2009-12-01 11:34:00,0.85,16329.0,United Kingdom
384,489517,22319,HAIRCLIPS FORTIES FABRIC ASSORTED,12,2009-12-01 11:34:00,0.65,16329.0,United Kingdom
385,489517,21913,VINTAGE SEASIDE JIGSAW PUZZLES,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
386,489517,21821,GLITTER STAR GARLAND WITH BELLS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
390,489517,84951A,S/4 PISTACHIO LOVEBIRD COASTERS,1,2009-12-01 11:34:00,2.55,16329.0,United Kingdom
391,489517,21491,SET OF THREE VINTAGE GIFT WRAPS,1,2009-12-01 11:34:00,1.95,16329.0,United Kingdom
394,489517,21912,VINTAGE SNAKES & LADDERS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
657,489529,22028,PENNY FARTHING BIRTHDAY CARD,12,2009-12-01 11:51:00,0.42,17984.0,United Kingdom
658,489529,22036,DINOSAUR BIRTHDAY CARD,12,2009-12-01 11:51:00,0.42,17984.0,United Kingdom


Note: After manual review, there are indeed lots of duplicated rows (e.g. row 371 and 394). Let's remove them.

In [8]:
retail = retail.drop_duplicates(keep='first')

In [9]:
#checking missing values
retail.isna().sum()

Invoice             0
StockCode           0
Description      4275
Quantity            0
InvoiceDate         0
Price               0
Customer ID    235151
Country             0
dtype: int64

Note: there are 234007 rows without Customer ID. Since we are working at the customer level, we cannot aggregate these columns. Let's remove them from further analysis.

In [10]:
retail.isna().sum()

Invoice             0
StockCode           0
Description      4275
Quantity            0
InvoiceDate         0
Price               0
Customer ID    235151
Country             0
dtype: int64

In [11]:
retail = retail[retail['Customer ID'].notna()]

In [12]:
retail.describe()

,Quantity,Price,Customer ID
count,797885.000000,797885.000000,797885.000000
mean,12.602980,3.702732,15313.062777
std,191.670371,71.392549,1696.466663
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13964.000000
50%,5.000000,1.950000,15228.000000
75%,12.000000,3.750000,16788.000000
max,80995.000000,38970.000000,18287.000000


Note: there is negative quantity in the dataset. Let's check it.

In [13]:
retail[retail.Quantity < 0].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.0,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.0,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321.0,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321.0,Australia


In [14]:
print(f"there are {len(retail[retail.Quantity < 0].Quantity)} rows with negative quantity.")

there are 18390 rows with negative quantity.


Note: Apparently returns. The invoices of these rows also start with C, which, according to description of the dataset, means cancellations. We want to keep this information.

In [15]:
# number of unique values in each column:
for col in retail.columns:
    print(f"{col}: {retail[col].nunique()}")

Invoice: 44876
StockCode: 4646
Description: 5299
Quantity: 643
InvoiceDate: 41439
Price: 1022
Customer ID: 5942
Country: 41


In [16]:
# According to specifications Stock Code needs to be 5 digits. Check for any non digit in Stock Code
retail[~retail.StockCode.str.match(pat='[0-9]{5}')].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
89,489439,POST,POSTAGE,3,2009-12-01 09:28:00,18.0,12682.0,France
126,489444,POST,POSTAGE,1,2009-12-01 09:55:00,141.0,12636.0,USA
173,489447,POST,POSTAGE,1,2009-12-01 10:10:00,130.0,12362.0,Belgium
625,489526,POST,POSTAGE,6,2009-12-01 11:50:00,18.0,12533.0,Germany
735,C489535,D,Discount,-1,2009-12-01 12:11:00,9.0,15299.0,United Kingdom


In [17]:
retail[~retail.StockCode.str.match(pat='[0-9]{5}')].StockCode.unique()

array(['POST', 'D', 'M', 'C2', 'BANK CHARGES', 'TEST001', 'TEST002',
       'PADS', 'ADJUST', 'ADJUST2', 'SP1002', 'DOT', 'CRUK'], dtype=object)

Note: there are many more mysterious Stock Codes than just 5 digit number. So after manual verification:<br>
'POST' = Postage <br>
'D' = Discount<br>
'M' = Manual<br>
'C2' = Carriage <br>
'BANK CHARGES' = Bank Charges <br>
'TEST001', 'TEST002' = test product  <br>
'PADS' = Pads to match all cushions <br>
'ADJUST' = Adjustment by john on 26/01/2010 16 <br>
'ADJUST2' = Adjustment by Peter on Jun 25 2010 <br>
'SP1002' = KID'S CHALKBOARD/EASEL <br>
'DOT' = DOTCOM postage <br>
'CRUK' = CRUK commission <br>

Apparently, we can exclude the following categories 'POST', 'C2', 'DOT', 'CRUK' and 'BANK CHARGES' because these are expenses for the company and they do not reflect the actual revenue streams. 
I decided to live other categories because they are either related to the products (test product, pads or SP1002) or revenue streams (discounts, adjustments, manual)

In [18]:
# excluding all the undesirable StockCode values 
exclude_test_idx = retail[retail.StockCode.str.contains("POST|C2|DOT|CRUK|BANK CHARGES", na=False, case=False)].index
retail = retail[~retail.index.isin(exclude_test_idx)]

In [19]:
#checking results
retail[~retail.StockCode.str.match(pat='[0-9]{5}')].StockCode.unique()

array(['D', 'M', 'TEST001', 'TEST002', 'PADS', 'ADJUST', 'ADJUST2',
       'SP1002'], dtype=object)

In [20]:
# checking types of columns
retail.dtypes

Invoice         object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
Price          float64
Customer ID    float64
Country         object
dtype: object

In [21]:
retail.InvoiceDate = pd.to_datetime(retail.InvoiceDate.astype(str), format="%Y-%m-%d %H:%M:%S")

In [115]:
# limit the data for one year 2011
retail_2011 = retail[retail.InvoiceDate.dt.year == 2011]

In [133]:
# checking the subset
retail_2011['InvoiceDate'].min(), retail_2011['InvoiceDate'].max()

(Timestamp('2011-01-04 10:00:00'), Timestamp('2011-12-09 12:50:00'))

## Step 2: RFM Model

In [ ]:
# find purchase size for each row
retail_2011.loc[:, 'PurchaseSize'] = retail_2011['Quantity']*retail_2011['Price']

In [144]:
# normally for recency metric today's date is used. but we will assign the last available date in the dataset as proxy
today = retail_2011['InvoiceDate'].max()

In [188]:
# aggregate data on customer level &
# calculate recency as the time elapsed since the last purchase by each customer till today,
# sum up all spending by each customer,
# count unique number of invoices per customer
rfm = retail_2011.groupby(['Customer ID']).agg({'InvoiceDate': lambda x: (today - x.max()).days,
                                                'Invoice': 'nunique',
                                                'PurchaseSize': sum})

In [189]:
#rename columns
rfm = rfm.rename(columns={'InvoiceDate': 'Recency', 'Invoice': 'Frequency', 'PurchaseSize': 'Monetary Value'})

In [190]:
rfm.head()

,Recency,Frequency,Monetary Value
Customer ID,,,
12346.0,325,2,0.00
12347.0,1,6,3598.21
12348.0,74,3,784.44
12349.0,18,1,1457.55
12350.0,309,1,294.40


## Step 3: Calculate RFM quartiles

In [191]:
# create labels in descending order - 4,3,2,1. Because the higher recency value, the worse.
# this means that the customer with the large recency value will be assigned to '1' quartile (bad)
rfm['R'] = pd.qcut(rfm['Recency'], q=4, labels=list(range(4, 0, -1)))

In [171]:
rfm.head()

,Recency,Frequency,Monetary Value,R
Customer ID,,,,
12346.0,325,2,0.00,1
12347.0,1,6,3598.21,4
12348.0,74,3,784.44,2
12349.0,18,1,1457.55,3
12350.0,309,1,294.40,1


In [192]:
# the higher frequency value, the better
# hence we create labels in ascending order from 1 to 4
# since we don't have unique bin edges here, we rank the values here to fix the problem
# possible issue - identical pre-ranking values might go to different quantiles
rfm['F'] = pd.qcut(rfm['Frequency'].rank(method='first'), q=4, labels=range(1,5))

In [196]:
# the higher, the better
rfm['M'] = pd.qcut(rfm['Monetary Value'], 4, labels=range(1,5))

In [197]:
rfm.head()

,Recency,Frequency,Monetary Value,R,F,M
Customer ID,,,,,,
12346.0,325,2,0.00,1,2,1
12347.0,1,6,3598.21,4,4,4
12348.0,74,3,784.44,2,2,3
12349.0,18,1,1457.55,3,1,3
12350.0,309,1,294.40,1,1,2


## Step 4: Build RFM Score

In [198]:
def join_rfm(x):
    'concatinates the R, F, M values into one string'
    return str(x['R']) + str(x['F']) + str(x['M'])

In [204]:
rfm['RFM_Segment'] = rfm.apply(join_rfm, axis=1)

In [205]:
rfm['RFM_Score'] = rfm[['R', 'F', 'M']].sum(axis=1)

In [258]:
rfm.head(10)

,Recency,Frequency,Monetary Value,R,F,M,RFM_Segment,RFM_Score,Custom_Segment
Customer ID,,,,,,,,,
12346.0,325,2,0.00,1,2,1,121,4.0,3_Low
12347.0,1,6,3598.21,4,4,4,444,12.0,1_Top
12348.0,74,3,784.44,2,2,3,223,7.0,2_Middle
12349.0,18,1,1457.55,3,1,3,313,7.0,2_Middle
12350.0,309,1,294.40,1,1,2,112,4.0,3_Low
12352.0,35,11,1265.41,3,4,3,343,10.0,1_Top
12353.0,203,1,89.00,1,1,1,111,3.0,3_Low
12354.0,231,1,1079.40,1,1,3,113,5.0,3_Low
12355.0,213,1,459.40,1,1,2,112,4.0,3_Low


## Step 5: Analyze RFM

### Top 10 RFM segments

In [224]:

rfm_segment_analysis = rfm.groupby(['RFM_Segment']).size().sort_values(ascending=False).reset_index().head(10)
rfm_segment_analysis = rfm_segment_analysis.rename(columns={0:'#_customers'})

In [230]:
rfm.Recency.count()

4244

In [233]:
rfm_segment_analysis.loc[:, 'pct'] = rfm_segment_analysis['#_customers'].divide(rfm.RFM_Segment.count()).round(3)*100

In [234]:
rfm_segment_analysis

,RFM_Segment,#_customers,pct
0,444,459,10.8
1,111,292,6.9
2,344,219,5.2
3,121,184,4.3
4,333,177,4.2
5,112,158,3.7
6,211,138,3.3
7,433,137,3.2
8,233,133,3.1
9,122,131,3.1


Note: 444 segment was the largest group of customers in 2011, which is good because it is also the best group possible. But it makes only about 11% of all customers. The second biggest segment is '111' (7%), which is also the worst group of customers. 

### Summary Metrics per RFM score

In [239]:
rfm.groupby('RFM_Score').agg({'Recency': 'mean', 'Frequency': 'mean', 'Monetary Value': ['mean', 'count']}).round(1)

Recency Frequency Monetary Value      
             mean      mean           mean count
RFM_Score                                       
3.0         237.7       1.0          132.2   292
4.0         187.7       1.2          225.0   480
5.0         118.4       1.4          332.7   505
6.0          87.3       1.8          474.4   487
7.0          70.6       2.4          663.9   431
8.0          55.4       3.1          948.7   380
9.0          40.5       4.2         1237.8   439
10.0         29.7       5.8         1815.7   386
11.0         18.0       8.9         3509.6   385
12.0          5.3      17.8         8422.4   459

In [254]:
def segment_custom(df):
    if df['RFM_Score'] >= 10.0:
        return '1_Top'
    if (df['RFM_Score'] >= 6.0) and (df['RFM_Score'] < 10.0):
        return '2_Middle'
    else:
        return '3_Low'

In [255]:
rfm['Custom_Segment'] = rfm.apply(segment_custom, axis=1)

In [256]:
rfm.groupby('Custom_Segment').agg({'Recency': 'mean', 'Frequency': 'mean', 'Monetary Value': ['mean', 'count']}).round(1)

Recency Frequency Monetary Value      
                  mean      mean           mean count
Custom_Segment                                       
1_Top             16.9      11.2         4811.4  1230
2_Middle          64.4       2.8          818.1  1737
3_Low            171.7       1.2          246.3  1277

## Credentials:

**Data Source:** Online Retail II Data Set, UCI Machine Learning Repository, http://archive.ics.uci.edu/ml/datasets/Online+Retail+II

Analysis done after completing the course "Customer Segmentation in Python" on DataCamp (instructor - 
Karolis Urbonas)